In [14]:
import numpy as np
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
import matplotlib.pyplot as plt
import time
import argparse

In [2]:
#upload data
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)

#split columns
df = pd.DataFrame(data[0].str.split(' ',9).tolist())
#extract one column --> blocktrace sequence
blocktrace = df[3].tolist()
#convert element to int from str
blocktrace = np.array([int(x) for x in blocktrace])

In [3]:
s = [0,1,2,0,1,3,0,3,1,2,1]
f = 3

In [4]:
frame = 3000

## FIFO

In [ ]:
def FIFO(blocktrace, frame):
    cache = np.zeros((frame), dtype=int)
    hit, miss = 0, 0

    #print("block", "\t\t hit", "\t\t miss", "\t\t cache", "\t\tpointer")

    pointer = 0
    for block in tqdm(blocktrace, desc=frame, leave=False):
        if block in cache:
            hit += 1

        elif pointer < frame:
            #cache[np.where(cache==0)[0][0]] = block
            cache[pointer] = block
            miss += 1
            pointer += 1

        else:
            cache[pointer % frame] = block
            pointer += 1
            miss += 1

        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache, "\t\t", pointer)

    hitrate = hit / (hit + miss)
    #print(hitrate)
    return hitrate

## LFU

In [22]:
def LFU(blocktrace, frame):
    cache = set()
    FU = defaultdict(int)
    FU_cache = defaultdict(int)

    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in tqdm(blocktrace, desc="LFU", leave=False):
        FU[block] += 1

        if block in cache:
            hit += 1
            FU_cache[block] = FU[block]
        
        elif len(cache) < frame:
            FU_cache[block] = FU[block]
            cache.add(block)
            miss += 1

        else:
            e, f = min(FU_cache.items(), key=lambda a: a[1])
            del FU_cache[e]
            cache.remove(e)
            cache.add(block)
            FU_cache[block] = FU[block]
            miss += 1
        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    #print("\n\nhitrate: ", hitrate)
    return hitrate

In [ ]:
LFU(blocktrace, 3000)

## LRU

In [23]:
def LRU(blocktrace, frame):
    cache = set()
    recent_block = np.array([], dtype=int)
    
    hit, miss = 0, 0
    
    #print("block", "\t\t hit", "\t\t miss", "\t\t cache")
    for block in tqdm(blocktrace, desc="LRU", leave=False):
    
        if block in cache:
            recent_block = np.delete(recent_block, np.argwhere(recent_block==block))
            recent_block = np.append(recent_block, block)
            hit += 1
            
        
        elif len(cache) < frame:
            recent_block = np.append(recent_block, block)
            cache.add(block)
            miss += 1

        else:
            cache.remove(recent_block[0])
            recent_block = recent_block[1:]
            recent_block = np.append(recent_block, block)
            cache.add(block)
            miss += 1
        #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

    hitrate = hit / (hit + miss)
    #print("\n\nhitrate: ", hitrate)
    return hitrate

In [59]:
LRU(blocktrace, 3000)

Exception ignored in: <function tqdm.__del__ at 0x000002435579B950>
Traceback (most recent call last):
  File "C:\Users\Jay\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 889, in __del__
    self.close()
  File "C:\Users\Jay\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 1095, in close
    self._decr_instances(self)
  File "C:\Users\Jay\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 454, in _decr_instances
    cls.monitor.exit()
  File "C:\Users\Jay\Anaconda3\lib\site-packages\tqdm\_monitor.py", line 52, in exit
    self.join()
  File "C:\Users\Jay\Anaconda3\lib\threading.py", line 1029, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread





hitrate:  0.05470598462457196


0.05470598462457196

In [57]:
LRU(s, f)


LRU: 100%|███████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 4436.28it/s]



hitrate:  0.5454545454545454


0.5454545454545454

## OPTIMAL

In [8]:
'''
get the furthest accessed block. Scans OPT dictionary and selects the maximum positioned element
'''

def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            #print ( "Not Acccessing block anymore " + str(cached_block))
            return cached_block            
    for cached_block in C:  
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    #print ( "chose to evict " + str(maxAccessBlock) + " since its position of access is " + str(maxAccessPosition))
    return maxAccessBlock

In [ ]:
def belady_opt(blocktrace, frame):
    OPT = defaultdict(partial(np.ndarray,0))

    for i, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        OPT[block] = np.append(OPT[block], i)    

    #print ("created OPT dictionary")    

    hit, miss = 0, 0

    C = set()

    seq_number = 0
    for block in tqdm(blocktrace, desc="OPT", leave=False):

        if block in C:
            #OPT[block] = OPT[block][1:]
            OPT[block] = np.delete(OPT[block],0)
            hit+=1
            #print('hit' + str(block))
            #print(OPT)
        else:
            #print('miss' + str(block))
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                C.remove(fblock)
            C.add(block)
            #OPT[block] = OPT[block][1:]
            #print(OPT)
            OPT[block] = np.delete(OPT[block],[0])

    #print ("hit count" + str(hit_count))
    #print ("miss count" + str(miss_count))
    hitrate = hit_count / (hit_count + miss_count)
    #print(hitrate)
    return hitrate

In [ ]:
belady_opt(blocktrace, 3000)

run OPT:  35%|██████████████████████▏                                        | 464748/1322890 [07:46<19:44, 724.69it/s]

In [11]:
frame_range = [3000, 5000, 10000, 50000, 100000]

In [ ]:
performance = {}

In [25]:
result = []
for frame in frame_range:
    t = time.time()
    ans = LRU(blocktrace, frame)
    result.append((ans, time.time() - t))
performance['LRU'] = result

In [ ]:
result = []
for frame in frame_range:
    t = time.time()
    ans = LFU(blocktrace, frame)
    result.append((ans, time.time() - t))
performance['LFU'] = result



hitrate:  0.05724814610436242




hitrate:  0.06510896597600707




hitrate:  0.08165153565300214




hitrate:  0.19543726235741446


In [16]:
result = []
for frame in frame_range:
    t = time.time()
    ans = FIFO(blocktrace, frame)
    result.append((ans, time.time() - t))
performance['FIFO'] = result

In [20]:
result = []
for frame in frame_range:
    t = time.time()
    ans = belady_opt(blocktrace, frame)
    result.append((ans, time.time() - t))
performance['OPT'] = result

-19.53780484199524